In [1]:
# Dependencies
import requests
import json
from pprint import pprint
import pandas as pd
import numpy as np
import re
from single_zip_function import get_schools_in_zip
from outliers_utility import single_year_outliers

# Import the API key
from config import geoapify_key

In [2]:
#set relevant year to search for outliers
my_year="2022"

#pull outliers from data
outliers_df = single_year_outliers(my_year)

In [3]:
#create list of zipcodes
zip_list = outliers_df["zip_code"].tolist()
zip_list = [str(int(z)) for z in zip_list if len(str(int(z))) == 5]

In [4]:
results=[] 

for zipcode in zip_list[:225]:   #change the number in zip_list parameter for how many schools to return
    print(zipcode)
    area_df, zip_df = get_schools_in_zip(zipcode)
    num_schools_in_area = len(area_df)
    num_schools_in_zip = len(zip_df)
    if num_schools_in_area == 0:
        lat = None
        lon = None
    else:
        lat = area_df.iloc[0]['lat']
        lon = area_df.iloc[0]['lon']
    results.append({'Zip Code': zipcode, 'Lat': lat, 'Lon': lon, 
                    'Schools in Area': num_schools_in_area,
                    'Schools in Zip Code': num_schools_in_zip})

49033
23840
47971
47963
47273
48649
24549
47220
48747
22853
47598
47633
22969
48615
24070
24067
48873
25401
35989
35978
35971
35962
35760
36110
36703
36260
36201
35087
35083
31028
34987
34972
31097
98923
35748
30641
35619
35228
35207
35224
98857
98858
38581
38562
39044
38751
38456
38356
39652
39573
39083
30182
39209
39213
37391
37737
37748
37352
37191
37342
38057
37853
37882
37863
33332
32466
32254
33498
33576
33573
33572
33437
33483
32811
32626
33160
32617
32603
33180
32668
33067
33579
34436
34429
99169
34286
34449
34285
31555
34229
34219
34210
34233
34266
34242
34275
34482
34748
34734
34950
34785
34759
34758
34653
34690
31321
98943
33647
32148
33711
32189
33873
33853
33850
33946
34202
33983
33955
33950
33947
33839
32113
33785
33812
45148
28339
28384
28467
44880
28470
44867
28462
45167
28018
27891
27880
45204
45214
44454
44450
44449
44438
44504
44307
44417
44402
44401
28636
28513
28604
44625
28635
45312
46057
27207
27043
27208
46340
27617
45402
45416
45337
45346
45845
27544
27559
4561

In [5]:
#return dataframe 

df = pd.DataFrame(results)
print(df)

    Zip Code        Lat         Lon  Schools in Area  Schools in Zip Code
0      49033  42.232265  -85.106929               20                    5
1      23840  37.079668  -77.581658               19                    0
2      47971  40.564264  -87.241869               17                    2
3      47963  40.943255  -87.451713               16                    2
4      47273  38.982488  -85.738818               18                    0
..       ...        ...         ...              ...                  ...
220    85138  33.059700 -111.954984               16                    8
221    85131  32.759848 -111.559993               11                    4
222    85118  33.342826 -111.427860               18                    3
223    85173  33.294886 -111.099216               19                    4
224    12067  42.577357  -73.953778               19                    0

[225 rows x 5 columns]


In [6]:
df.to_csv('results.csv', index=False)